# <center> GPT-Neo 2.7B - Open Form Q&A with Colab </center>

Uses EleutherAI's text generation model + huggingface *Transformers* library pipeline feature for an easy-to-use means of working with the model.

---

## about

1. **Model Description**

    GPT-Neo 2.7B is a transformer model designed using EleutherAI's replication of the GPT-3 architecture. GPT-Neo refers to the class of models, while 2.7B represents the number of parameters of this particular pre-trained model.

2. **Training data**

    GPT-Neo 2.7B was trained on the Pile, a large scale curated dataset created by EleutherAI for the purpose of training this model.

3. **Training procedure**

    This model was trained for 420 billion tokens over 400,000 steps. It was trained as a masked autoregressive language model, using cross-entropy loss.

4. **Intended Use and Limitations**

    This way, the model learns an inner representation of the English language that can then be used to extract features useful for downstream tasks. The model is best at what it was pretrained for however, which is generating texts from a prompt.

## links
- [link](https://huggingface.co/EleutherAI/gpt-neo-2.7B) to transformers website, [docs](https://huggingface.co/transformers/model_doc/gpt_neo.html) for GPT-Neo model
- [link](https://github.com/EleutherAI/gpt-neo) to eleutherAI github repo
- As of 22.06.2021 the model is not yet on transformers, but there is a 6B-parameter version [here](https://6b.eleuther.ai/) for testing

## Note

- <font color="salmon"> *Before running make sure that the Colab Runtime is set to a high-ram GPU. If only standard-memory is available, the notebook will adjust to a downsized model* 

---


In [ ]:
!nvidia-smi # shows GPU status - preferably want 12 gb or higher here

Mon Aug 30 02:16:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup

## make colab outputs nice

In [ ]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## install libraries

In [ ]:
%%capture
!pip install -U transformers
!pip install clean-text[gpl]
!pip install GPUtil

from transformers import pipeline
from cleantext import clean
import GPUtil

import pprint as pp
import os, gc

## clean-text helper function

fixes a lot of the ```\n``` outputs and so on generated by the model

In [ ]:
def clean_gpt_out(text, remove_breaks=True):
    cleaned_text = clean(text,
                         fix_unicode=True,               # fix various unicode errors
                        to_ascii=True,                  # transliterate to closest ASCII representation
                        lower=False,                     # lowercase text
                        no_line_breaks=remove_breaks,           # fully strip line breaks as opposed to only normalizing them
                        no_urls=True,                  # replace all URLs with a special token
                        no_emails=True,                # replace all email addresses with a special token
                        no_phone_numbers=True,         # replace all phone numbers with a special token
                        no_numbers=False,               # replace all numbers with a special token
                        no_digits=False,                # replace all digits with a special token
                        no_currency_symbols=True,      # replace all currency symbols with a special token
                        no_punct=False,                 # remove punctuations
                        replace_with_punct="",          # instead of removing punctuations you may replace them
                        replace_with_url="<URL>",
                        replace_with_email="<EMAIL>",
                        replace_with_phone_number="<PHONE>",
                        replace_with_number="<NUMBER>",
                        replace_with_digit="0",
                        replace_with_currency_symbol="<CUR>",
                        lang="en"                       # set to 'de' for German special handling
                    )
    return cleaned_text


## load model

### check gpu

In [ ]:
!nvidia-smi
# printed device ID is relevant for running on GPU (I.e. 0)

Mon Aug 30 02:17:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import logging 
import numpy as np
LOGGER = logging.getLogger()
def gpuname():
    # Returns the model name of the first available GPU
    try:
        gpus = GPUtil.getGPUs()
    except:
        LOGGER.warning("Unable to detect GPU model. Is your GPU configured? Is Colab Runtime set to GPU?")
        return "UNKNOWN"
    if len(gpus) == 0:
        raise ValueError("No GPUs detected in the system")
    return gpus[0].name 

def gpu_mem_total():
    # Returns the total memory of the first available GPU
    try:
        gpus = GPUtil.getGPUs()
    except:
        LOGGER.warning("Unable to detect GPU model. Is your GPU configured? Is Colab Runtime set to GPU?")
        return np.nan
    if len(gpus) == 0:
        raise ValueError("No GPUs detected in the system")
    return gpus[0].memoryTotal 

get the RAM of the accompanying operating CPU

In [ ]:
# Getting all memory using os.popen() cpu_
cpu_total_memory, cpu_used_memory, cpu_free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])

cpu_RAM_tot = round(cpu_total_memory / 1024, 2)
print(cpu_RAM_tot, cpu_used_memory, cpu_free_memory)

51.0 909 37163


### load from hf hub

details on how to configure a pipeline are [here](https://huggingface.co/transformers/v3.0.2/main_classes/pipelines.html)

_NOTE: the [official EleutherAI release](https://huggingface.co/EleutherAI/gpt-j-6B) on HF does not seem to work, hence using a duplicate also posted on HF here._

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM



model_6B_pars = "flyhero/gpt-j-6B" # see note above
model_3B_pars = 'EleutherAI/gpt-neo-2.7B'
model_1B_pars = "EleutherAI/gpt-neo-1.3B"

gpu_mem = round(gpu_mem_total() / 1024, 2)

if gpu_mem > 17 and cpu_RAM_tot > 36:
    print("using biggest 6B model. GPU - {} GB, CPU-RAM - {} GB".format(gpu_mem,
                                                                    cpu_RAM_tot))
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    # actual_model = AutoModelForCausalLM.from_pretrained(model_6B_pars)
    generator = pipeline('text-generation', model=model_6B_pars, 
                         tokenizer=tokenizer, device=0) 

elif gpu_mem > 14 and cpu_RAM_tot > 16:
    print("using medium model. GPU - {} GB, CPU-RAM - {} GB".format(gpu_mem,
                                                                    cpu_RAM_tot))
    actual_model = model_3B_pars
    generator = pipeline('text-generation', model=actual_model, 
                     device=0) 
else:
    actual_model = model_1B_pars
    print("using SMALLER model. GPU - {} GB, CPU-RAM - {} GB".format(gpu_mem,
                                                                    cpu_RAM_tot))
    print("using the smaller {} model".format(actual_model))
    generator = pipeline('text-generation', model=actual_model, 
                     device=0) 

gc.collect()

using medium model. GPU - 15.78 GB, CPU-RAM - 51.0 GB


Downloading:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

995

# Test Model

## baseline test

- while the output is relatively random each time (seed is not fixed), can get a feel for how reasonable the model is.
- "*Nikola Tesla, (born July 9/10, 1856, Smiljan, Austrian Empire [now in Croatia]—died January 7, 1943, New York, New York, U.S.), Serbian American inventor and engineer who discovered and patented the rotating magnetic field, the basis of most alternating-current machinery.*" - [Britannica](https://www.britannica.com/biography/Nikola-Tesla)

In [ ]:
generator("Nikola Tesla was born on", do_sample=True, min_length=50)

[{'generated_text': 'Nikola Tesla was born on August 18, 1856 in Kostin, a small village in Macedonia, on the Ottoman Empire. Tesla was the only child in his family. From the beginning of Tesla’s life he developed a passion for'}]

## text completion / story generation 

(edit items in the form to customize)

In [ ]:
prompt1 = "I opened my eyes, and immediately" #@param {type:"string"}
response_min_chars =  100#@param {type:"integer"}
response_max_chars =  500#@param {type:"integer"}
import pprint as pp
response1 = generator(prompt1, do_sample=True, min_length=response_min_chars, 
                      max_length=response_max_chars,
                      clean_up_tokenization_spaces=True,
                      return_full_text=True)
gc.collect()
print("Prompt: \n")
pp.pprint(prompt1)
print("\nResponse: \n")
out1_dict = response1[0]
pp.pprint(clean_gpt_out(out1_dict["generated_text"]), compact=True)

Prompt: 

'I opened my eyes, and immediately'

Response: 

('I opened my eyes, and immediately a black fog engulfed the room. I sat up in '
 'bed and saw all the curtains moving around with the wind, and the branches '
 'and leaves flapping in the wind, making a sound that I had never heard '
 'before. Everything seemed to stop. The room was completely silent. '
 'Everything was silent. I heard a slight noise from the bathroom, where I had '
 'just showered, and heard the sound of running water in the shower, and then '
 'nothing. I lay down again and fell asleep, but woke up again, just before '
 'noon. This time I sat up, in bed, and listened, but no one was there. I '
 'tried to open the curtains to see if anything was there, but the wind had '
 "carried them away, and I couldn't see anything. I went to the bathroom and "
 'looked into the mirror. And there... in the reflection... was a tiny man, '
 "his head sticking out the bathroom door, and I couldn't find his head "
 'anywhere 

## text completion / Q&A 


(edit items in the form to customize)

In [ ]:
prompt2 = "the easiest way to become a Swiss citizen as a foreigner is" #@param {type:"string"}
response_min_chars =  100#@param {type:"integer"}
response_max_chars =  300#@param {type:"integer"}

response_2 = generator(prompt2, do_sample=True, min_length=response_min_chars, 
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
gc.collect()
print("Prompt: \n")
pp.pprint(prompt2)
print("\nResponse: \n")
out2_dict = response_2[0]
pp.pprint(clean_gpt_out(out2_dict["generated_text"], 
                        remove_breaks=True), compact=True)

Prompt: 

'the easiest way to become a Swiss citizen as a foreigner is'

Response: 

('the easiest way to become a Swiss citizen as a foreigner is just to give '
 'birth. The Swiss allow non-EU applicants to apply as a single person in '
 'cases where their marriage does not fulfill all formal requirements, and '
 'where the applicant is the sole financial breadwinner for their household. '
 'When applying for the single person classification of citizenship, '
 'applicants may show that their parents were citizens of Switzerland, at '
 "least on their parent's birth and death certificates. The Swiss citizenship "
 'law has been amended several times since 2003 to allow naturalisation of '
 'non-EU citizens who are married to a Swiss citizen who was not resident in '
 'Switzerland at the time of the marriage. In 2019, the Swiss Supreme Court '
 'confirmed the legality of the process. Non-EU citizens who were born after 1 '
 'January 2004 may apply for Swiss citizenship as a single perso

## direct Q&A 

(edit items in the form to customize)

In [ ]:
prompt3 = "question: what is the meaning of life?" #@param {type:"string"}
response_min_chars =  100#@param {type:"integer"}
response_max_chars =  500#@param {type:"integer"}

response_3 = generator(prompt3, do_sample=True, min_length=response_min_chars, 
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
gc.collect()
print("Prompt: \n")
pp.pprint(prompt3)
print("\nResponse: \n")
out3_dict = response_3[0]
pp.pprint(clean_gpt_out(out3_dict["generated_text"], 
                        remove_breaks=True), compact=True)

Prompt: 

'question: what is the meaning of life?'

Response: 

('question: what is the meaning of life? how does it evolve? what is the '
 'meaning of death? the answers to these questions are really very difficult '
 "questions, I know. i also know that there's lot of theories that explain the "
 'meaning of life. for example, according to what i know in the last 25 years, '
 'what we know now is that life is a very short period of time. this means '
 'that life really is very short and it really is very easy to be trapped in '
 'our life. this is so because we are so much dependent on our surroundings. '
 'according to the science of life, life is a product of life. we can use the '
 'science of life to define our purpose in life. we can also use the science '
 'of life to redefine our purpose in life. so here are the answers to your '
 'question, what the meaning of life is, life is a very short period of time, '
 'and it really is very easy to be trapped inside your life. we are s

## idea generation 

(edit items in the form to customize)

In [ ]:
prompt4 = "ideas for an app that predicts where new construction sites will be built:" #@param {type:"string"}
response_min_chars =  100#@param {type:"integer"}
response_max_chars =  500#@param {type:"integer"}

response_4 = generator(prompt4, do_sample=True, min_length=response_min_chars, 
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
gc.collect()
print("Prompt: \n")
pp.pprint(prompt4)
print("\nResponse: \n")
out4_dict = response_4[0]
pp.pprint(clean_gpt_out(out4_dict["generated_text"], 
                        remove_breaks=True), compact=True)

Prompt: 

'ideas for an app that predicts where new construction sites will be built:'

Response: 

('ideas for an app that predicts where new construction sites will be built: '
 "_I'm excited to see what this app gives me. I wish it could have told me "
 'whether a site would be better for the community or a better place for new '
 'residents._ **_Example question_** _This question is open-ended, allowing '
 'you to select a topic that may not even apply to your app. This question is '
 "about how your app would feel when it's trying to predict a city's housing "
 "market. You may use this question to develop your app's capabilities._ "
 '**_Example answer_** _This question is a question about your app using '
 'current housing market data to predict future housing market activity._ '
 '**_Example discussion_** _This discussion is an open discussion on a '
 'question that your app may or may not have answered._ **_Example decision '
 'tree_** _Here is a decision tree that you could h

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.1 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, Trainer, TrainingArguments

In [3]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)
# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>',
                            force_download=True)
# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Embedding(50259, 768)

In [4]:
descriptions = pd.read_csv('/content/sample_data/docQA_train.csv')['data']
print(len(list(descriptions)))
#descriptions = pd.read_csv('https://www.kaggle.com/datasets/shivamb/netflix-shows')
max_length = max([len(tokenizer.encode(description)) for description in descriptions])

147


In [5]:
class QADataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>' 
                                        + txt +    
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_id = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_id)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [6]:
dataset = QADataset(descriptions, tokenizer, max_length)
print(len(dataset))

147


In [7]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])


In [8]:
# Here I will pass the output directory where 
# the model predictions and checkpoints will be stored, 
# batch sizes for the training and validation steps, 
# and warmup_steps to gradually increase the learning rate
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=5,
                                  logging_steps=5000,
                                  save_steps=5000,                                   
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2,
                                  warmup_steps=100,
                                  weight_decay=0.01,  
                                  logging_dir='./logs')

In [9]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset, 
                  #num_samples = 50,
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])})
# Start training process!
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 132
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 330
  Number of trainable parameters = 125200128


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=330, training_loss=4.735395581794507, metrics={'train_runtime': 244.7473, 'train_samples_per_second': 2.697, 'train_steps_per_second': 1.348, 'total_flos': 361292269916160.0, 'train_loss': 4.735395581794507, 'epoch': 5.0})

In [28]:
ocrt = " BankStatements.net BANKOFAMERICA/ Business Advantage P.O. Box 15284 Wilmington, DE 19850 Customer service information 1.888.BUSINESS (1.888.287.4637) bankofamerica.com MR JOHN DOE Bank of America, N.A 2 POST ALLEY, P.O. Box 25118 SEATTLE, WA 98101 Tampa, FL 33622-5118  Please see the Important Messages - Please Read section of your statement for important details that could impact you. Your Business Fundamentals Checking for February 1, 2021 to February 28, 2021 Account number: 1 2345 6789 KC UNLOCKING COMPANY Account summary Beginning balance on February 1, 2021 $39.65 # of deposits/credits: 28 Deposits and other credits 24,983.78 # of withdrawals/debits: 43 Withdrawals and other debits -24,139.29 # of items-previous cycle1: 2 Checks -0.00 # of days in cycle: 28 Service fees -70.00 Average ledger balance: $1,091.41 Ending balance on February 28, 2021 $814.14 1Includes checks paid, deposited items & other debits BUSINESS ADVANTAGE Thanks. We're here to listen to you. As your business need evolve, we're ready to provide personal attention and access to the latest digital tools. Rely on us for guidance in personal finance, investments and busines - now and in the future. Page 1 of 11"
qt = "What is the Statement Period?"
txt = "<|startoftext|> [CONTEXT]: "+ocrt+"\n"+"[QUESTION]: "+qt+"\n"+"[ANSWER]: "

In [29]:
# Start every description with a special BOS token
generated = tokenizer(txt,   
                      return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, 
                 # Use sampling instead of greedy decoding 
                 do_sample=True, 
                 # Keep only top 50 token with 
                 # the highest probability
                 top_k=50, 
                 # Maximum sequence length
                 max_length=300, 
                 # Keep only the most probable tokens 
                 # with cumulative probability of 95%
                 top_p=0.95, 
                 # Changes randomness of generated sequences
                 temperature=1.9,
                 # Number of sequences to generate                 
                 num_return_sequences=20)
# Print generated descriptions
#for i, sample_output in enumerate(sample_outputs): 
print("{}: {}".format(0, tokenizer.decode(sample_outputs[0], skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 339, but `max_length` is set to 300. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


0:  [CONTEXT]:  BankStatements.net BANKOFAMERICA/ Business Advantage P.O. Box 15284 Wilmington, DE 19850 Customer service information 1.888.BUSINESS (1.888.287.4637) bankofamerica.com MR JOHN DOE Bank of America, N.A 2 POST ALLEY, P.O. Box 25118 SEATTLE, WA 98101 Tampa, FL 33622-5118  Please see the Important Messages - Please Read section of your statement for important details that could impact you. Your Business Fundamentals Checking for February 1, 2021 to February 28, 2021 Account number: 1 2345 6789 KC UNLOCKING COMPANY Account summary Beginning balance on February 1, 2021 $39.65 # of deposits/credits: 28 Deposits and other credits 24,983.78 # of withdrawals/debits: 43 Withdrawals and other debits -24,139.29 # of items-previous cycle1: 2 Checks -0.00 # of days in cycle: 28 Service fees -70.00 Average ledger balance: $1,091.41 Ending balance on February 28, 2021 $814.14 1Includes checks paid, deposited items & other debits BUSINESS ADVANTAGE Thanks. We're here to listen to you. As